# 連接至資料庫抓資料，並套用 MySQL 的 DAYOFYEAR、WEEK、WEEKDAY ... 日期函式將日期轉為需要的格式

In [1]:
# 匯入需要的檔案 [B]
%matplotlib inline
import time
import matplotlib.pyplot as plt
import numpy as np # Use numpy to convert to arrays
import pandas as pd # 引用套件並縮寫為 pd  
import seaborn as sns
import  pymysql
# 匯入需要的檔案 [E]

In [24]:
# 連接至資料庫，並抓取資料 [B]
conn  =  pymysql.connect ( host = ' ' ,  user = ' ' ,  passwd = " " ,  db = ' ' ) 
cur  =  conn.cursor() 
select_sql = '''SELECT `d`.`id`, `d`.`datetime`, DAYOFYEAR(`d`.`datetime`) AS `day_of_year`,
                WEEK(`d`.`datetime`) AS `data_week`, WEEKDAY(`d`.`datetime`) AS `data_weekday`, DATE(`d`.`datetime`) AS `data_date`, 
                HOUR(`d`.`datetime`) AS `data_hour`,MINUTE(`d`.`datetime`) AS `data_minute`,
                TIME_TO_SEC(`d`.`datetime`)/60 AS `minutes_of_the_day`,
                `d`.`demand_min`, `d`.`demand_quarter`, `d`.`Total_value`, 
                `wd`.`Temperature`, `wd`.`T_Max`, `wd`.`T_Min`
                FROM `demand` AS `d`
                INNER JOIN `weather_data` AS `wd` ON(DATE(`d`.`datetime`) = DATE(`wd`.`T_Max_Time`))
                WHERE (`datetime` LIKE \'%:15:00\' 
                      OR `datetime` LIKE \'%:30:00\' 
                      OR `datetime` LIKE \'%:45:00\' 
                      OR `datetime` LIKE \'%:00:00\') 
                AND `model` = \'DEM600-4\'
                ORDER BY `d`.`datetime`'''
result_object = cur.execute(select_sql)
results_values_list = cur.fetchall()
result_key_list = [i[0] for i in cur.description]
cur.close () 
conn.close()
# 連接至資料庫，並抓取資料 [E]

In [25]:
# 將抓取到的資料轉為 dataframe [B]
df = pd.DataFrame(results_values_list)
df.columns = result_key_list
df
# 將抓取到的資料轉為 dataframe [E]

,id,datetime,day_of_year,data_week,data_weekday,data_date,data_hour,data_minute,minutes_of_the_day,demand_min,demand_quarter,Total_value,Temperature,T_Max,T_Min
0,162,2017-06-08 13:45:00,159,23,3,2017-06-08,13,45,825.0000,258.0,283.6,254.0,30.0,34.1,27.9
1,252,2017-06-08 14:00:00,159,23,3,2017-06-08,14,0,840.0000,264.0,277.6,268.0,30.0,34.1,27.9
2,342,2017-06-08 14:15:00,159,23,3,2017-06-08,14,15,855.0000,270.0,269.6,286.0,30.0,34.1,27.9
3,432,2017-06-08 14:30:00,159,23,3,2017-06-08,14,30,870.0000,294.0,288.4,301.0,30.0,34.1,27.9
4,522,2017-06-08 14:45:00,159,23,3,2017-06-08,14,45,885.0000,300.0,295.2,289.0,30.0,34.1,27.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119828,10771391,2021-02-05 14:30:00,36,5,4,2021-02-05,14,30,870.0000,144.0,134.0,138.0,19.1,23.6,15.4
119829,10771481,2021-02-05 14:45:00,36,5,4,2021-02-05,14,45,885.0000,132.0,137.6,131.0,19.1,23.6,15.4
119830,10771571,2021-02-05 15:00:00,36,5,4,2021-02-05,15,0,900.0000,126.0,144.4,127.0,19.1,23.6,15.4
119831,10771661,2021-02-05 15:15:00,36,5,4,2021-02-05,15,15,915.0000,132.0,143.2,129.0,19.1,23.6,15.4
